In [1]:
from PIL import Image

In [9]:
img = Image.open('input00.jpg').convert('L')
img = img.point(lambda x: 0 if x<50 else 255, '1')
img.save('input00_noise_removed.png')
img

In [7]:
type(img)

PIL.Image.Image

In [37]:
p = Image.open('input00_noise_removed.png').convert('P')
p

In [27]:
w, h = p.size
w, h

(60, 30)

In [42]:
up, down = 0, 0
for j in range(h):
    found = False
    for i in range(w):
        if p.getpixel((i, j)) == 0:
            up = j
            found = True
            break
    if found:
        break
for j in range(h-1, -1, -1):
    found = False
    for i in range(w):
        if p.getpixel((i,j)) == 0:
            found = True
            down = j
            break
    if found:
        break    
up, down

(11, 20)

In [47]:
def remove_noise(img_path):
    img = Image.open(img_path).convert('L')
    img = img.point(lambda x: 0 if x<50 else 255, '1')
    img_name = img_path.split('/')[-1]
    img.save('noise-removed/{}'.format(img_name))

In [48]:
from glob import glob

all_img_paths = glob('input/*.jpg')
all_img_paths
for img_path in all_img_paths:
    remove_noise(img_path)

In [79]:
def cut_and_crop(img_path):
    img = Image.open(img_path).convert('P')
    global part
    letters = []
    left, right = 0, 0
    w,h, up, down = 60, 30, 11, 21
    found = False
    for i in range(w):
        in_letter = False
        for j in range(h):
            if img.getpixel((i,j)) == 0:
                in_letter = True
                break
        if not found and in_letter:
            found = True
            left = i
        if found and not in_letter and i-left>6:
            found = False
            right = i
            letters.append((left, right))
    for left, right in letters:
        bbox = (left, up, right, down)
        crop = img.crop(bbox)
        crop.save('crop/part_{}.png'.format(part))
        part += 1
        
def cut_and_crop2(img_path):
    img = Image.open(img_path).convert('P')
    global part
    bbox = (5, 11, 13, 21)
    crop = img.crop(bbox)
    crop.save('crop/part_{}.png'.format(part))
    part += 1
    bbox = (14, 11, 22, 21)
    crop = img.crop(bbox)
    crop.save('crop/part_{}.png'.format(part))
    bbox = (23, 11, 31, 21)
    crop = img.crop(bbox)
    crop.save('crop/part_{}.png'.format(part))
    bbox = (32, 11, 40, 21)
    crop = img.crop(bbox)
    crop.save('crop/part_{}.png'.format(part))
    bbox = (41, 11, 49, 21)
    crop = img.crop(bbox)
    crop.save('crop/part_{}.png'.format(part))

In [81]:
import os
from shutil import rmtree

part = 0
all_img_paths = glob('noise-removed/*.jpg')
if os.path.exists('crop'):
    rmtree('crop')
os.mkdir('crop')
for img_path in all_img_paths:
    cut_and_crop(img_path)

In [89]:
from PIL import ImageChops

def detect_char(img_path):
    char_paths = glob('chars/*.png')
    class Fit:
        letter = None
        difference = 0
    best = Fit()
    _img = Image.open(img_path)
    for char_path in char_paths:
        current = Fit()
        img = Image.open(char_path)
        current.letter = char_path.split('/')[-1].split('.')[0]
        difference = ImageChops.difference(_img, img)
        for x in range(difference.size[0]):
            for y in range(difference.size[1]):
                current.difference += difference.getpixel((x, y))/255.
        if not best.letter or best.difference > current.difference:
            best = current
    print(img_path, best.letter)
#     if best.letter == filename.split('-')[0]: return
#     print(filename, best.letter)
#     rename(path, filename, best.letter)

In [93]:
detect_char('part_4.png')

part_4.png 4


In [98]:
def checker(img_path):
    img = Image.open(img_path).convert('P')
    letters = []
    left, right = 0, 0
    w,h, up, down = 60, 30, 11, 21
    found = False
    for i in range(w):
        in_letter = False
        for j in range(h):
            if img.getpixel((i,j)) == 0:
                in_letter = True
                break
        if not found and in_letter:
            found = True
            left = i
        if found and not in_letter and i-left>6:
            found = False
            right = i
            letters.append((left, right))
    for left, right in letters:
        print(left, right, up, down)

img_path = 'noise-removed/input00.jpg'
checker(img_path)

5 12 11 21
14 22 11 21
23 31 11 21
32 40 11 21
42 49 11 21


In [99]:
checker('noise-removed/input01.jpg')

5 13 11 21
14 22 11 21
23 31 11 21
33 40 11 21
41 49 11 21


In [100]:
checker('noise-removed/input03.jpg')

6 13 11 21
14 21 11 21
23 31 11 21
32 40 11 21
41 49 11 21
